In [3]:
import boto3

# Use AWS CLI named profile
session = boto3.Session(profile_name="bivlab")
sagemaker_runtime = session.client("sagemaker-runtime")
s3_client = session.client("s3")

In [4]:
ENDPOINT_NAME = "tensorflow-inference-2024-07-11-09-00-17-984"
ALLOWED_EXTENSIONS = {"png", "jpg", "jpeg", "gif"}

In [5]:
import io
import json
import urllib.parse
import numpy as np
from PIL import Image

In [6]:
def preprocess_image(image_content):
    img = Image.open(image_content)
    img = img.resize((150, 150))
    img = np.array(img)

    # Ensure that the image has three channels (for RGB images)
    if img.shape[-1] != 3:
        img = img[:, :, :3]  # Keep only the first three channels

    img = img / 255.0  # Normalize the image
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

def make_prediction(bucket_name, image_key):
    # Check if the file extension is allowed
    extension = image_key.rsplit(".", 1)[1].lower()

    if extension not in ALLOWED_EXTENSIONS:
        return {
            "bucket": bucket_name,
            "file": image_key,
            "label": "INVALID FILE TYPE",
            "score": 0,
        }

    # Retrieve the uploaded file content
    response = s3_client.get_object(Bucket=bucket_name, Key=image_key)
    file_data = response["Body"].read()
    image_content = io.BytesIO(file_data)
    input_data = preprocess_image(image_content)

    # Send the file content to the SageMaker endpoint for prediction
    response = sagemaker_runtime.invoke_endpoint(
        EndpointName=ENDPOINT_NAME,
        ContentType="application/json",
        Body=json.dumps(input_data.tolist()).encode(),
    )

    # Process the prediction result
    response_body = json.loads(response["Body"].read().decode())
    result = response_body["predictions"][0][0]
    label = "GOOD WELD" if result > 0.8 else "BAD WELD"

    return {
        "bucket": bucket_name,
        "file": image_key,
        "label": label,
        "score": result
    }

In [7]:
def lambda_handler(event, context):
    results = []

    for record in event["Records"]:
        bucket = record["s3"]["bucket"]["name"]
        key = urllib.parse.unquote_plus(record["s3"]["object"]["key"], encoding="utf-8")
        result = make_prediction(bucket, key)
        results.append(result)

    print(results)
    return {"statusCode": 200, "body": json.dumps(results)}

In [8]:
event_bad = {
    "Records": [
        {
            "s3": {
                "bucket": {"name": "banpu-interns-data-exchange"},
                "object": {"key": "data/test/bad_weld/augmented_1607.png"},
            }
        },
        {
            "s3": {
                "bucket": {"name": "banpu-interns-data-exchange"},
                "object": {"key": "data/test/good_weld/augmented_1639.png"},
            }
        },
    ]
}

lambda_handler(event_bad, None)

[{'bucket': 'banpu-interns-data-exchange', 'file': 'data/test/bad_weld/augmented_1607.png', 'label': 'BAD WELD', 'score': 3.98490919e-12}, {'bucket': 'banpu-interns-data-exchange', 'file': 'data/test/good_weld/augmented_1639.png', 'label': 'GOOD WELD', 'score': 1.0}]


{'statusCode': 200,
 'body': '[{"bucket": "banpu-interns-data-exchange", "file": "data/test/bad_weld/augmented_1607.png", "label": "BAD WELD", "score": 3.98490919e-12}, {"bucket": "banpu-interns-data-exchange", "file": "data/test/good_weld/augmented_1639.png", "label": "GOOD WELD", "score": 1.0}]'}